# 마포구청 모범음식점 크롤링
- url 4곳에서 음식정 정보 크롤링
- 동일한 구조로 되어있는 table 데이터 별도의 dataframe으로 확보
- 각각 json 형식 파일로 내보내기

In [1]:
!pwd

/Users/hanul/Dev/mapojoy


In [2]:
# urls
urls = ['https://www.mapo.go.kr/site/culture/mo/onstop_food_mo_list',
        'https://www.mapo.go.kr/site/culture/cl/onstop_food_cl_list',
        'https://www.mapo.go.kr/site/culture/tu/onstop_food_tu_list',
        'https://www.mapo.go.kr/site/culture/an/onstop_food_an_list']
mapo_url = 'https://www.mapo.go.kr/site/culture/mo/onstop_food_mo_list'

In [3]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

마포구청 음식점 사이트 열기

In [4]:
driver = webdriver.Chrome()

driver.get(mapo_url)

# implicitly wait for n seconds
driver.implicitly_wait(2.5)

# scroll down some
driver.execute_script("window.scrollTo(0, 1000)") 

In [5]:
# 음식점 row 읽어서 저장하는 부분
def get_store_info(driver):
    stores = []
    # 한 페이지의 음식점 정보 한번에 row에 담기
    tbody = driver.find_element(By.CSS_SELECTOR, "tbody")
    rows = tbody.find_elements(By.CSS_SELECTOR, "tr")
    
    for row in rows:
        row_tds = row.find_elements(By.CSS_SELECTOR, 'td')
        record = [td.text for td in row_tds]
        try:
            record[-1] = row.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        except:
            record[-1] = '-'
        
        stores.append(record)
    return stores
    

## urls만큼 반복

In [6]:
for url in urls:
    driver.get(url)

    # implicitly wait for n seconds
    time.sleep(1.5)

    # scroll down some
    driver.execute_script("window.scrollTo(0, 1000)") 
    
    # 페이지 데이터 가져오기 (1회)
    page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
    page_info = page_info[2:]
    
    # page 이동

    now = 0
    prev_first_page = -1
    stores = []

    # paginatoin flag
    is_page_ended = False

    # 첫 페이지 별도로 진행
    res = get_store_info(driver)
    for r in res:
        stores.append(r)
    
    while (True):
    
        time.sleep(1.2)
        print("현재 페이지: " , now+1)
        

        if now == (len(page_info) - 2 -1):
            now = 0
        
            print("페이지 이동합니다!", page_info[0].text)
            prev_first_page = int(page_info[0].text)
            # print("이동 전 맨 첫페이지:", prev_first_page)
        
            # 다음페이지로 가기
            page_info[-2].click()
            time.sleep(1)
        
            # link 다시 받아오기
            page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
            page_info = page_info[2:]
        
            # 다음페이지의 맨 첫 아이템과 현 페이지 가장 첫 아이템이 같으면 플래그 풀어주기
            if int(page_info[0].text) == prev_first_page:
                break
        
        
        else:
            page_info[now+1].click()
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 900)") 
            time.sleep(1)
            now += 1

        # link 다시 받아오기
        page_info = driver.find_elements(By.CSS_SELECTOR, ".bbs_paging a") # 맨처음/이전/숫자 1~10/다음/맨끝
        page_info = page_info[2:]
        print("\t페이지수: ", len(page_info) - 2)
    
    
        # wait for full load
        time.sleep(1.5)
    
        # ========= 음식점 정보 받아오는 부분 ==========
        temp = get_store_info(driver)
        for t in temp:
            stores.append(t)
    
    storeDf = pd.DataFrame(stores, columns = ['번호', '식당명', '유형', '주메뉴', '위치', '전화번호', '홈페이지'])
    filename = url.split('/')[-1]
    storeDf.to_json(f'./mapojoy123_{filename}.json', orient="records", force_ascii = False)
    
    print(f"===================\n{url} 페이지 크롤링 끝!================\n")
    
print("===================\n전체 크롤링 종료!!===================\n")
    
    

현재 페이지:  1
	페이지수:  10
현재 페이지:  2
	페이지수:  10
현재 페이지:  3
	페이지수:  10
현재 페이지:  4
	페이지수:  10
현재 페이지:  5
	페이지수:  10
현재 페이지:  6
	페이지수:  10
현재 페이지:  7
	페이지수:  10
현재 페이지:  8
	페이지수:  10
현재 페이지:  9
	페이지수:  10
현재 페이지:  10
페이지 이동합니다! 1
	페이지수:  2
현재 페이지:  1
	페이지수:  2
현재 페이지:  2
페이지 이동합니다! 11
https://www.mapo.go.kr/site/culture/mo/onstop_food_mo_list 페이지 크롤링 끝!================

현재 페이지:  1
	페이지수:  3
현재 페이지:  2
	페이지수:  3
현재 페이지:  3
페이지 이동합니다! 1
https://www.mapo.go.kr/site/culture/cl/onstop_food_cl_list 페이지 크롤링 끝!================

현재 페이지:  1
	페이지수:  3
현재 페이지:  2
	페이지수:  3
현재 페이지:  3
페이지 이동합니다! 1
https://www.mapo.go.kr/site/culture/tu/onstop_food_tu_list 페이지 크롤링 끝!================

현재 페이지:  1
	페이지수:  10
현재 페이지:  2
	페이지수:  10
현재 페이지:  3
	페이지수:  10
현재 페이지:  4
	페이지수:  10
현재 페이지:  5
	페이지수:  10
현재 페이지:  6
	페이지수:  10
현재 페이지:  7
	페이지수:  10
현재 페이지:  8
	페이지수:  10
현재 페이지:  9
	페이지수:  10
현재 페이지:  10
페이지 이동합니다! 1
	페이지수:  10
현재 페이지:  1
	페이지수:  10
현재 페이지:  2
	페이지수:  10
현재 페이지:  3
	페이지수:  10
현재 페이지:  4
	페이지수:  10
현재 페이지

In [7]:
driver.quit()